# Práctico 4 - Content-based (Texto)

Diplomado ML Aplicado

**Profesor:** Vicente Domínguez

**Nombre del alumno: Sebastian Latorre**


En este proyecto trabajaremos con un modelo de recomendacion de libros de la página [Goodreads](http://www.goodreads.com). El modelo de recomendación de libros es un recomendador basado en contenido, donde se utilizan modelos de lenguage BERT y BERT-large para el cálculo de embeddings de los libros y luego similaridades de ítems. Luego, dependiendo de los libros con los que el usuario ha interactuado, se recomiendan los ítems más similares.

In [1]:
import numpy as np
import json
import requests
import heapq
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from io import BytesIO
import pickle
import pandas as pd
import time

Descargamos datos que vienen previamente calculados:
- transacciones/interacciones de cada usuario
- transaciones para evaluar el modelo
- embeddings de descripciones calculados con BERT  
- embeddings de descripciones calculados con BERT-large
- datos de libros con información de titulo, descripcion, año de publicacion, entre otros.

In [2]:
%%capture
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

In [3]:
ls

books.csv                            goodreads_past_interactions.json
goodreads_bert_embeddings.npy        goodreads_test_interactions.json
goodreads_bert_large_embeddings.npy  sample_data/


## Todos los libros del catálogo de goodreads

In [4]:
df_books = pd.read_csv('books.csv', sep=',')
df_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...


## Interacciones pasadas de los usuarios (ej. libros leídos)

In [5]:
# diccionario con id del usuario y id de libros con los que ha interactuado en el pasado
with open('goodreads_past_interactions.json') as f:
    user_interactions = json.load(f)

# diccionario con id del usuario y id de libros para testear el modelo
with open('goodreads_test_interactions.json') as f:
    user_interactions_test = json.load(f)


In [6]:
# dict index 2 book id and vice-versa for recommendation
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

In [7]:
np.load('goodreads_bert_embeddings.npy').shape

(4287, 768)

In [8]:
np.load('goodreads_bert_large_embeddings.npy').shape

(4287, 1024)

# Cargar características pre-entrenadas: BERT y BERT-large

En esta sección se trabajará con modelos pre-entrenados de modelos de lenguage BERT y BERT-large que convierten texto a embeddings.

Bidirectional Encoder Representations from Transformers (BERT) es una técnica de NLP (Natural Language Processing) desarrollada por Google y publicada en 2018 por Jacob Devlin.

Actualmente Google utiliza BERT para entender las consultas de los usuarios en su buscador.

Tiene dos versiones:
- **BERT:** 12 capas, 12 cabezales de atencion y 110 millones de parámetros. Genera vectores de 768 dimensiones
- **BERT-large:** 24 capas, 16 cabezales de atencion y 340 millones de parámetros.  

![BERT y BERT-large](http://jalammar.github.io/images/bert-base-bert-large.png)

![BERT y BERT-large arquitectura](http://jalammar.github.io/images/bert-base-bert-large-encoders.png)

En este caso los textos que utilizaremos son los títulos de los libros con su descripción y compararemos los resultados de recomendación con BERT y BERT-large. Para efectos de este trabajo los vectores de características ya fueron entrenados y guardados en archivos numpy. A continuación son cargados en memoria.

Para mayores detalles sobre el modelo de lenguaje BERT se recomienda revisar el siguiente artículo:
- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)


In [9]:
# cargamos los embeddings de BERT y de BERT-large
bert_featmat = np.load('goodreads_bert_embeddings.npy', allow_pickle=True)
bert_large_featmat = np.load('goodreads_bert_large_embeddings.npy', allow_pickle=True)

In [10]:
bert_featmat.shape # BERT son vectores de 768 dimensiones para cada libro (4287)

(4287, 768)

In [11]:
bert_large_featmat.shape # BERT-large son vectores de 1024 dimensiones para cada libro (4287)

(4287, 1024)

### **Pregunta 1**

Considerando que haremos un recomendador basado en contenido ¿Por qué el uso de modelos de lenguage es una buena elección para este tipo de problema?

**Respuesta 1:**

`RESPONDER AQUI`


```
El uso de modelos de lenguaje BERT es una buena elección para un recomendador
basado en contenido por varias razones:

*   Comprensión Profunda del Lenguaje Natural: Los modelos de lenguaje como
BERT están diseñados para entender y procesar el lenguaje natural a un nivel
profundo, capturando tanto el contexto como el significado de las palabras en
un texto. Esto permite que las descripciones y títulos de los libros sean
representados de manera más precisa en un espacio de características.

*   Embeddings Contextuales: A diferencia de los métodos tradicionales de
procesamiento de texto, BERT genera embeddings contextuales, es decir, la
representación de una palabra depende del contexto en el que se encuentra.
Esto es crucial para capturar las sutilezas y el significado real de las
descripciones de los libros.

*   Versatilidad y Robustez: Los embeddings generados por BERT pueden ser
utilizados directamente para calcular similitudes entre ítems, lo que facilita
la creación de recomendaciones basadas en contenido. Además, estos embeddings
son robustos y pueden adaptarse bien a diferentes tipos de texto.
```




# Probamos con BERT y BERT-large reduciendo dimensionalidad con PCA a 20 dimensiones.

Una vez calculado (o cargado) los vectores característicos de cada libro a partir de su titulo y descripción, reducimos dimensionalidad. Probaremos con BERT y BERT-large para comparar los resultados de ambos en recomendación basada en contenido.


In [12]:
# Project into a 20 PCA feature space
pca20_bert_featmat = PCA(n_components=20).fit_transform(bert_featmat)
pca20_bert_large_featmat = PCA(n_components=20).fit_transform(bert_large_featmat)

In [13]:
pca20_bert_featmat.shape

(4287, 20)

In [14]:
pca20_bert_large_featmat.shape

(4287, 20)

### **Pregunta 2**

Comente por qué si se utiliza PCA para reducir la dimensión de cada vector a 20 la velocidad (tiempo) de hacer recomendación puede disminuir?

### Respuesta 2:

`RESPONDER AQUI`



```
Utilizar PCA para reducir la dimensión de cada vector a 20 puede disminuir la
velocidad (tiempo) de hacer recomendaciones por las siguientes razones:

Reducción de Complejidad Computacional: Al reducir la cantidad de dimensiones
de los embeddings de 768 (BERT) o 1024 (BERT-large) a solo 20, se disminuye
significativamente la cantidad de cálculos necesarios para medir las
similitudes entre los ítems. Las operaciones de distancia (como la distancia
coseno) se vuelven más rápidas con vectores de menor dimensión.

Menor Carga de Memoria: Menos dimensiones significan menos datos para almacenar
y manipular, lo que reduce la carga de memoria y acelera los tiempos de acceso
y procesamiento.

Mejora en la Eficiencia de Algoritmos de Búsqueda: Algoritmos que dependen de
la dimensionalidad de los datos, como el uso de heaps para encontrar los ítems
más similares, se benefician de la menor dimensionalidad, resultando en tiempos
de ejecución más cortos.
```



# Recomendaciones

In [15]:
# format results
pd.options.display.max_colwidth = 50
#pd.set_option('display.max_colwidth', -1)

In [16]:
def recommend(embedding, user_id=None, topk=10, metric='cosine'):

    user_id = str(user_id)

    #Calculate distance metrics
    trx = user_interactions[user_id]
    n = embedding.shape[0]
    distances = 1e9

    # recorremos transacciones pasadas del usuario
    for t in trx:
        query_i = bookid2idx[t]

        # recomendamos items más cercanos a items con los que interactuó el usuario
        distances = np.minimum(distances, pairwise_distances(
                embedding[query_i].reshape(1,-1), embedding, metric=metric).reshape(-1))

    #Rank items de menor a mayor distancia (nos quedamos con los topk)
    trx_set = set(trx)
    heap = []

    for i in range(n):
        if idx2bookid[i] in trx_set:
            continue
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[i], i))
        else:
            heapq.heappushpop(heap, (-distances[i], i))

    heap.sort(reverse=True)

    # utilizamos un heap para extraer los items ordenados de menor a mayor distancia
    recommended_ids = [idx2bookid[i] for _,i in heap]

    # retornar los que el usuario no haya consumido
    filtered_recommended_ids = []

    return recommended_ids

## generar recomendaciones para un usuario en particular

In [17]:
import random

random.sample(user_interactions.keys(), 10) # vemos 10 usuarios aleatorios

<ipython-input-17-1b4254b22161>:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(user_interactions.keys(), 10) # vemos 10 usuarios aleatorios


['6324',
 '48437',
 '8914',
 '21839',
 '430',
 '2632',
 '26802',
 '34779',
 '47205',
 '9368']

In [18]:
# recomendación para el usuario id = 49299 , utilizando bert con reduccion de dimensionalidad a 20
user_id = '32204'
rec = recommend(pca20_bert_featmat, user_id=user_id, topk=15)
rec

[9785,
 6265,
 7032,
 10,
 6250,
 6865,
 3349,
 9941,
 7701,
 5244,
 48,
 5126,
 2847,
 390,
 2796]

## transacciones pasadas del usuario

In [19]:
past_interactions = user_interactions[str(user_id)]

df_books[df_books['book_id'].isin(past_interactions)][['book_id', 'original_title']]


,book_id,original_title
22,26,The Da Vinci Code
50,61,The Girl on the Train
91,114,Tuesdays with Morrie
96,119,The Handmaid's Tale
99,123,The Firm
...,...,...
2610,4961,The Fifth Season
2615,4972,Straight Man
2849,5578,The Nix
4138,9504,The Obelisk Gate


## información de recomendaciones

In [20]:
df_books[df_books['book_id'].isin(rec)][['book_id', 'original_title']]

,book_id,original_title
8,10,Pride and Prejudice
39,48,Fahrenheit 451
293,390,The Strange Case of Dr Jekyll and Mr Hyde
1643,2796,The Lost Wife
1673,2847,La Vérité sur l'affaire Harry Quebert
1921,3349,Se questo è un uomo
2676,5126,Travesuras de la niña mala
2718,5244,Il barone rampante
3105,6250,Soumission
3111,6265,L’écume des jours


# Evaluación de las recomendaciones

In [21]:
# Métricas de evaluación
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg


## Evaluación de recomendación con BERT

In [22]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(bert_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [23]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))


MAP  0.03669047619047619
ndcg@10 0.13
tiempo de ejecucion 395.41 segs


## Evaluación de recomendación con BERT reducidos con PCA-20

In [24]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(pca20_bert_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [25]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.00453968253968254
ndcg@10 0.03
tiempo de ejecucion 27.02 segs


## Evaluación de recomendación con BERT-large

In [26]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start

In [27]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.060920634920634924
ndcg@10 0.24
tiempo de ejecucion 609.90 segs


## Evaluación de recomendación con BERT-large reducidos con PCA-20

In [28]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(pca20_bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start

In [29]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.026123015873015874
ndcg@10 0.07
tiempo de ejecucion 27.47 segs


### Pregunta 3:
- Comente los resultados en términos de tiempo de ejecución y métricas de ranking para los 4 modelos. (BERT, BERT-large, BERT reducido con PCA a 20 dimensiones , BERT-large reducido con PCA a 20 dimensiones)

### Respuesta 3:

`RESPONDER AQUI`

```
Los resultados en tiempo de ejecución y métricas
de ranking para los 4 modelos son los siguientes:

*   BERT:
MAP  0.03669047619047619
ndcg@10 0.13
tiempo de ejecucion 395.41 segs

*   BERT reducido con PCA a 20 dimensiones:
MAP  0.00453968253968254
ndcg@10 0.03
tiempo de ejecucion 27.02 segs

*   BERT-large:
MAP  0.060920634920634924
ndcg@10 0.24
tiempo de ejecucion 609.90 segs

*   BERT-large reducido con PCA a 20 dimensiones:
MAP  0.026123015873015874
ndcg@10 0.07
tiempo de ejecucion 27.47 segs

Comentarios sobre los resultados:

Precisión y Calidad de Recomendación:
*   Los modelos sin reducción de dimensionalidad (BERT y BERT-large)
tienen mejores métricas de ranking (MAP y NDCG) comparados con sus
versiones reducidas con PCA.
*   BERT-large presenta las mejores métricas de ranking, indicando
que captura mejor las similitudes entre los libros.

Tiempo de Ejecución:
*   Los modelos reducidos con PCA (tanto BERT como BERT-large) muestran
una mejora significativa en el tiempo de ejecución, siendo alrededor de
10 veces más rápidos que los modelos sin reducción de dimensionalidad.
*   La reducción de dimensionalidad a 20 componentes con PCA resulta en una
pérdida de precisión en las recomendaciones, pero esta pérdida puede ser
ceptable dependiendo del caso de uso y la necesidad de velocidad en el
sistema de recomendación.

En resumen, hay un trade-off entre precisión y velocidad. Para aplicaciones
donde la velocidad es crítica, usar PCA puede ser beneficioso, mientras que
para aplicaciones donde la precisión es más importante, es mejor usar los
embeddings completos.
```

